In [ ]:
a = set(['A','B','C'])
b = set(['B','A','D'])
c = set(['C','A','H'])
d = set(['D','B','E'])
e = set(['E','D','G'])
f = set(['F','G'])
g = set(['G','E','F'])
h = set(['H','C'])

Consider it as :
'1': 'A',
'2': 'B', '3': 'C', '4': 'D', '5': 'E', '6': 'F', '7': 'G', '8': 'H'

1.1)
Integer Programming

**Objective:**
Minimize
$$ \sum_{i=1}^{n} x_i $$

**Constraints:**
$$ \sum_{i=1}^{n} t_{ij} x_i \leq 25, \quad \forall j=1,2,...,n $$

$$ \sum_{j=1}^{n} t_{ij} x_j \leq 25, \quad \forall i=1,2,...,n $$

**Decision Variables:**
$$ x_i \in \{0,1\}
 \quad \forall i=1,2,...,n $$



Using Pyomo

In [ ]:
#The city needs to build minimum number of fire stations needed to ensure there is atleast one fire station within 25 minutes (driving time) of each location

In [ ]:
import pandas as pd
import numpy as np

In [ ]:
!pip install -q pyomo
from pyomo.environ import *

# Install solver as well into collab session
!apt-get install -y -qq glpk-utils

In [ ]:
model = ConcreteModel()

In [ ]:
from typing import cast
import networkx as nx
import numpy as np
import pyomo.core as pyo
from IPython.display import Markdown, display
from matplotlib import pyplot as plt

In [ ]:
import itertools
from typing import List

import pyomo.core as pyo


def set_cover(sub_sets: List[List[int]]) -> pyo.ConcreteModel:
    entire_set = set(itertools.chain(*sub_sets))
    n = max(entire_set)
    num_sets = len(sub_sets)
    assert entire_set == set(
        range(1, n + 1)
    ), f"the union of the subsets is {entire_set} not equal to range(1, {n + 1})"

    model = pyo.ConcreteModel()
    model.x = pyo.Var(range(num_sets), domain=pyo.Binary)

    @model.Constraint(entire_set)
    def independent_rule(model, num):
        return sum(model.x[idx] for idx in range(num_sets) if num in sub_sets[idx]) >= 1

    model.cost = pyo.Objective(expr=sum(model.x.values()), sense=pyo.minimize)

    return model

In [ ]:
sub_sets = [
    [1, 2, 3],
    [1,2, 4],
    [1,3,8],
    [2,5,4],
    [5,7,4],
    [6,7],

]


In [ ]:
set_cover_model = set_cover(sub_sets)


In [ ]:
set_cover_model.pprint()

In [ ]:
s=SolverFactory('glpk', executable='/usr/bin/glpsol').solve(model)

In [ ]:
from pyomo.opt import SolverFactory

solver = SolverFactory("glpk")
solver.solve(set_cover_model)

set_cover_model.display()

In [ ]:
classical_solution = [
    pyo.value(set_cover_model.x[i]) for i in range(len(set_cover_model.x))
]

In [ ]:
print(
    f"Classical Solution: num_sets={int(sum(classical_solution))}, sets={[sub_sets[i] for i in range(len(classical_solution)) if classical_solution[i]]}"
)

The 1st fire station can be placed at C location

The 2nd fire station can be placed at E location

The 3rd fire station can be placed at G location

** Set Cover Problem Argument:**

In this scenario, think of each spot as a point that needs attention, and the groups represent potential spots where fire stations could be created. The time limit for travel serves as the condition that each point must be within reach of at least one group within 25 minutes. So essentially, we're dealing with a set covering issue where the goal is to reduce the number of groups (fire stations) required to encompass all points (locations).

In [ ]:
def greedy_set_cover(locations, fire_stations, driving_times):
    uncovered_locations = set(locations)
    selected_stations = []

    while uncovered_locations:
        best_station = None
        max_covered = -1

        for station in fire_stations:
            covered = set()
            for location in uncovered_locations:
                if driving_times[location][station] <= 25:
                    covered.add(location)
            if len(covered) > max_covered:
                max_covered = len(covered)
                best_station = station

        selected_stations.append(best_station)
        uncovered_locations -= set([best_station])

    return selected_stations


The greedy algorithm for set cover doesn't always guarantee an optimal solution.
It makes locally optimal choices at each step without global consideration.
Strategic sacrifices for long-term efficiency may be overlooked.
Different evaluation orders can yield varying, possibly suboptimal, solutions.

**1.4**

When we needed 20 minutes of driving time

In [ ]:
sub_sets2 = [
    [1, 2],
    [2, 4,1],
    [1,3,8],
    [4,2],
    [5,7],
    [5,6,7]

]


In [ ]:
set_cover_model2 = set_cover(sub_sets2)

In [ ]:
set_cover_model2.pprint()

In [ ]:
from pyomo.opt import SolverFactory

solver = SolverFactory("glpk")
solver.solve(set_cover_model2)

set_cover_model2.display()

In [ ]:
classical_solution = [
    pyo.value(set_cover_model.x[i]) for i in range(len(set_cover_model2.x))
]

In [ ]:
print(
    f"Classical Solution: num_sets={int(sum(classical_solution))}, sets={[sub_sets2[i] for i in range(len(classical_solution)) if classical_solution[i]]}"
)

5)

In [ ]:
def greedy_set_cover(collection_of_subsets, universe):
    uncovered_elements = set(universe)
    selected_subsets = []

    while uncovered_elements:
        best_subset = None
        max_covered = -1

        for subset in collection_of_subsets:
            covered = set()
            for element in uncovered_elements:
                if element in subset:
                    covered.add(element)
            if len(covered) > max_covered:
                max_covered = len(covered)
                best_subset = subset

        selected_subsets.append(best_subset)
        uncovered_elements -= best_subset

    return selected_subsets

coll_of_subsets = [{"1","2"},{"2","4"},{"3","8"},{"5","7"},{"6","7"}]
Universe = set(['1', '2', '3', '4', '5', '6', '7', '8'])

fire_station_at_6 = [{'6'}]
remaining_locations = Universe - set('6')

sol_remaining = greedy_set_cover(coll_of_subsets, remaining_locations)

additional_fire_stations = len(sol_remaining)

data = {'Fire Station': [], 'Location': []}
for i, station in enumerate(sol_remaining):
    location = list(station)[0]
    data['Fire Station'].append(f"Station {i+2}")
    data['Location'].append(location)

num_to_letter = {'1': 'A', '2': 'B', '3': 'C', '4': 'D', '5': 'E', '6': 'F', '7': 'G', '8': 'H'}
df = pd.DataFrame(data)
df['Location'] = df['Location'].map(num_to_letter)
print(df)
